In [ ]:
import pandas as pd

# Define the path to the Excel file
file_path = "study_permits_by_country_2015_2025.xlsx"

try:
    # Read the Excel file with specific parameters:
    # skiprows=2: Skip the first 2 rows. These rows contain introductory information
    #            or headers that are not part of the main data structure.
    # skipfooter=7: Skip the last 7 rows (footer). These rows contain
    #             summary statistics, notes, or other information not needed for the analysis.
    # header=[0, 1, 2]: Set the header as the rows at index 0, 1, and 2 (after skipping the initial rows).
    #                  These three rows together form the MultiIndex for the column names,
    #                  representing Year, Quarter, and Month/Total.
    df = pd.read_excel(file_path, skiprows=2, skipfooter=7, header=[0, 1, 2])

    # Function to flatten the MultiIndex columns into a single string
    def flatten_multiindex(col):
        if isinstance(col, tuple):
            # Join the parts of the tuple with ' - ' if they are not 'Unnamed'.
            # 'Unnamed' indicates merged cells in the original Excel sheet
            # that do not contain specific header information at that level.
            return ' - '.join(str(x) for x in col if 'Unnamed' not in str(x))
        return col

    # Apply the flatten_multiindex function to the columns to create simpler names
    df.columns = df.columns.map(flatten_multiindex)

    # Rename the first column, which contains the country names.
    # The original MultiIndex for this column spanned three rows with 'Unnamed' levels.
    df.rename(columns={'Country of Citizenship - Unnamed: 0_level_1 - Unnamed: 0_level_2': 'Country of Citizenship'}, inplace=True)

    # Display the cleaned column names to verify the transformation
    print("Cleaned Column Names:")
    print(df.columns)

    # Display the first few rows of the cleaned DataFrame to inspect the data structure
    print("\nFirst few rows of Cleaned DataFrame:")
    print(df.head())

except FileNotFoundError:
    # Handle the error if the specified file path does not exist
    print(f"Error: File '{file_path}' not found. Please check the path.")
except Exception as e:
    # Handle other potential errors that might occur during file reading
    print(f"An error occurred while reading the file: {e}")

In [ ]:
# Identify columns containing 'Total'
columns_to_drop = [col for col in df.columns if 'Total' in col]

# Drop the identified columns
df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')

# Display the remaining column names
print("\nColumn names after removing 'Total' columns:")
print(df_cleaned.columns)

# Display the first few rows of the cleaned DataFrame
print("\nFirst few rows of DataFrame after removing 'Total' columns:")
print(df_cleaned.head())

In [ ]:
# Melt the DataFrame to long format
df_long = pd.melt(df_cleaned,
                  id_vars=['Country of Citizenship'],
                  var_name='Period',
                  value_name='Number of Students')

# Display the first few rows of the long format DataFrame
print("\nFirst few rows of DataFrame in long format (df_long):")
print(df_long.head())

# Display the shape of the long format DataFrame
print("\nShape of the long format DataFrame (df_long):", df_long.shape)

In [ ]:
# Function to extract Year, Quarter, and Month from the 'Period' column
def extract_period_info(period_str):
    parts = period_str.split(' - ')
    year = int(parts[0])
    quarter = parts[1]
    month = parts[2]
    return pd.Series([year, quarter, month])

# Apply the function to create new 'Year', 'Quarter', and 'Month' columns
df_long[['Year', 'Quarter', 'Month']] = df_long['Period'].apply(extract_period_info)

# Display the first few rows of df_long with the new columns
print("\nFirst few rows of df_long with 'Year', 'Quarter', 'Month' columns:")
print(df_long)

# Display the data types of the new columns
print("\nData types of 'Year', 'Quarter', 'Month' columns:")
print(df_long[['Year', 'Quarter', 'Month']].dtypes)

In [ ]:
# Replace '--' with 0 in 'Number of Students'
df_long['Number of Students'] = df_long['Number of Students'].replace('--', 0)

# Remove commas and convert to integer
df_long['Number of Students'] = df_long['Number of Students'].astype(str).str.replace(',', '', regex=False)
df_long['Number of Students'] = pd.to_numeric(df_long['Number of Students'], errors='coerce').astype(int)

# Display the data type and first few rows
print("\nData type of 'Number of Students' column after cleaning and conversion:")
print(df_long['Number of Students'].dtype)
print("\nFirst few rows of df_long:")
print(df_long.head())

In [ ]:
# Group by 'Year' and sum the 'Number of Students'
yearly_permits = df_long.groupby('Year')['Number of Students'].sum()

# Display the total number of study permits per year
print("\nTotal number of study permits issued per year:")
print(yearly_permits)

# A simple line plot to visualize the trend
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(yearly_permits.index, yearly_permits.values, marker='o')
plt.title('Total Study Permits Issued in Canada (2015-2025)')
plt.xlabel('Year')
plt.ylabel('Number of Study Permits')
plt.grid(True)
plt.xticks(yearly_permits.index.astype(int)) # Ensure integer year labels
plt.tight_layout()

# Add annotation for 2025 data being partial
plt.annotate(
    'Note: 2025 data is partial (Jan only)',
    xy=(2025, yearly_permits.loc[2025]),  # Point to the 2025 data point
    xytext=(2023, yearly_permits.loc[2025] * 1.1),  # Position of the text
    arrowprops=dict(facecolor='black', shrink=0.05),
    fontsize=10,
    ha='right',
    va='bottom'
)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Group by 'Country of Citizenship' and sum the 'Number of Students'
country_permits = df_long.groupby('Country of Citizenship')['Number of Students'].sum()

# Sort the results in descending order to see the top countries
top_countries = country_permits.sort_values(ascending=False)

plt.figure(figsize=(14, 8))
top_countries.head(10).plot(kind='bar', color='skyblue')

plt.title('Top 10 Countries for Study Permits in Canada (2015-2025)', fontsize=16)
plt.xlabel('Country of Citizenship', fontsize=12)
plt.ylabel('Total Number of Study Permits', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10) 
plt.yticks(fontsize=10)

# Format y-axis to include commas for thousands
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.grid(axis='y', linestyle='--') # Added a horizontal grid

plt.tight_layout()
plt.show()


In [ ]:
# Get the top 5 countries
top_n = 5
top_n_countries = top_countries.head(top_n).index.tolist()

# Filter df_long to include only the top N countries
df_top_n = df_long[df_long['Country of Citizenship'].isin(top_n_countries)]

# Group by 'Year' and 'Country of Citizenship' and sum the 'Number of Students'
yearly_permits_top_n = df_top_n.groupby(['Year', 'Country of Citizenship'])['Number of Students'].sum().unstack()

# Create a line plot of the trends for the top N countries
plt.figure(figsize=(12, 7))
for country in yearly_permits_top_n.columns:
    plt.plot(yearly_permits_top_n.index, yearly_permits_top_n[country], marker='o', label=country)

plt.title(f'Study Permit Trends for the Top {top_n} Countries (2015-2025)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Study Permits', fontsize=12)
plt.xticks(yearly_permits_top_n.index.astype(int), rotation=45, ha='right')
plt.legend(title='Country', fontsize=10)
plt.grid(True, linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Define the order of months
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Convert 'Month' column to a categorical type with the specified order
df_long['Month'] = pd.Categorical(df_long['Month'], categories=month_order, ordered=True)

# Group by 'Month' and sum the 'Number of Students'
monthly_permits = df_long.groupby('Month')['Number of Students'].sum()

# Create a bar chart of the monthly distribution
plt.figure(figsize=(12, 6))
monthly_permits.plot(kind='bar', color='skyblue')
plt.title('Distribution of Study Permits by Month', fontsize=14)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Number of Study Permits', fontsize=12)
plt.xticks(rotation=0, ha='center')  # Ensure month names are horizontal
plt.tight_layout()
plt.show()

In [ ]:
# Group by 'Year' and sum the 'Number of Students'
yearly_permits = df_long.groupby('Year')['Number of Students'].sum()

# Calculate the year-over-year percentage change in study permits
yearly_permits_pct_change = yearly_permits.pct_change() * 100

# Display the year-over-year percentage change
print("\nYear-over-year percentage change in study permits:")
print(yearly_permits_pct_change)

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(yearly_permits_pct_change.index, yearly_permits_pct_change.values, marker='o', color='lightcoral')
plt.title('Year-over-Year Percentage Change in Study Permits (2016-2025)', fontsize=14)
plt.xlabel('Year', fontsize=12)
plt.ylabel('Percentage Change (%)', fontsize=12)
plt.xticks(yearly_permits_pct_change.index.astype(int), rotation=45, ha='right')
plt.axhline(0, color='black', linewidth=0.8)
plt.grid(True, linestyle='--')
plt.tight_layout()

# Add annotation for 2025
plt.annotate(
    '2025 data is partial (Jan only)',
    xy=(2025, yearly_permits_pct_change.loc[2025]),  # Point to the 2025 data point
    xytext=(2023, yearly_permits_pct_change.loc[2025] * .8),  # Adjust y-position as needed
    arrowprops=dict(facecolor='black', shrink=0.05),
    fontsize=10,
    ha='right',
    va='bottom'
)

plt.show()